In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection  import train_test_split 
from sklearn.metrics import accuracy_score, log_loss, mean_squared_error
import json

In [2]:
# Connect to a cluster
#port_no=random.randint(5555,55555)
port_no = 54321
#  h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o
h2o.init(strict_version_check=False) # start h2o

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 24.80-b11, mixed mode)
  Starting server from C:\Users\deodh\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: c:\users\deodh\appdata\local\temp\tmpfnaikh
  JVM stdout: c:\users\deodh\appdata\local\temp\tmpfnaikh\h2o_deodh_started_from_python.out
  JVM stderr: c:\users\deodh\appdata\local\temp\tmpfnaikh\h2o_deodh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_deodh_7cp5l2
H2O cluster total nodes:,1
H2O cluster free memory:,3.535 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
df=pd.read_csv("mushrooms.csv")

In [4]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
df.columns

Index([u'class', u'cap-shape', u'cap-surface', u'cap-color', u'bruises',
       u'odor', u'gill-attachment', u'gill-spacing', u'gill-size',
       u'gill-color', u'stalk-shape', u'stalk-root',
       u'stalk-surface-above-ring', u'stalk-surface-below-ring',
       u'stalk-color-above-ring', u'stalk-color-below-ring', u'veil-type',
       u'veil-color', u'ring-number', u'ring-type', u'spore-print-color',
       u'population', u'habitat'],
      dtype='object')

In [6]:
df.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [7]:
df.dtypes

class                       object
cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
dtype: object

In [8]:
df = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
df.columns

[u'class',
 u'cap-shape',
 u'cap-surface',
 u'cap-color',
 u'bruises',
 u'odor',
 u'gill-attachment',
 u'gill-spacing',
 u'gill-size',
 u'gill-color',
 u'stalk-shape',
 u'stalk-root',
 u'stalk-surface-above-ring',
 u'stalk-surface-below-ring',
 u'stalk-color-above-ring',
 u'stalk-color-below-ring',
 u'veil-type',
 u'veil-color',
 u'ring-number',
 u'ring-type',
 u'spore-print-color',
 u'population',
 u'habitat']

# Function for Run id

In [10]:
def run_id(n):
    letter='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
    str=''
    r=len(letter)-1   
    while len(str)<n:
        i=random.randint(0,r)
        str+=letter[i]   
    return str
server_path=None

In [11]:
data_path= 'C:\Users\deodh\Desktop\hyperparameter\mushrooms.csv'
all_variables=None
test_path=None
target=None
nthreads=1 
min_mem_size=6 
run_time=200
classification=True
#scale=False
max_models=100   
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
#analysis=0

In [12]:

def meta_data(run_id,server,data,test,model_path,target,run_time,regression,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
    m_data={}
    m_data['start_time'] = time.time()
    m_data['target']=target
    m_data['server_path']=server
    m_data['data_path']=data 
    m_data['test_path']=test
    m_data['max_models']=model
    m_data['run_time']=run_time
    m_data['run_id'] =run_id
    m_data['scale']=scale
    m_data['classification']=classification
    m_data['scale']=False
    m_data['model_path']=model_path
    m_data['balance']=balance
    m_data['balance_threshold']=balance_threshold
    m_data['project'] =name
    m_data['end_time'] = time.time()
    m_data['execution_time'] = 0.0
    m_data['run_path'] =path
    m_data['nthreads'] = nthreads
    m_data['min_mem_size'] = min_mem_size
   # m_data['analysis'] = analysis
    return m_data

In [13]:
target = 'class'

def get_independent_variables(train_data, targ):
    C = [name for name in train_data.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in train_data.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x = ints + enums + reals
    return x

X = get_independent_variables(df, target) 
print(X)
y = target
print(y)

[u'veil-color', u'cap-surface', u'habitat', u'odor', u'stalk-root', u'cap-shape', u'cap-color', u'stalk-color-above-ring', u'spore-print-color', u'gill-color', u'population', u'stalk-color-below-ring', u'ring-type', u'stalk-shape', u'bruises', u'stalk-surface-above-ring', u'veil-type', u'gill-attachment', u'gill-spacing', u'ring-number', u'stalk-surface-below-ring', u'gill-size']
class


In [14]:
# setup autoML
# min_mem_size=6 
run_time=200
aml1 = H2OAutoML(max_runtime_secs=run_time)

In [15]:
model_start_time = time.time()
aml1.train(x=X,y=y,training_frame=df)  # Change training_frame=train

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:

runid=run_id(10)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,runid)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (runid)

In [16]:
# view leaderboard
lb1 = aml1.leaderboard
lb1.head(500)

model_id,auc,logloss,mean_per_class_error,rmse,mse
GLM_grid_1_AutoML_20190423_030857_model_1,1,0.00199809,0,0.00783831,6.1439e-05
GBM_5_AutoML_20190423_030857,1,1.54084e-13,0,2.27237e-12,5.16365e-24
GBM_1_AutoML_20190423_030857,1,1.14931e-17,0,8.04002e-16,6.46419e-31
GBM_grid_1_AutoML_20190423_030857_model_1,1,0.0693869,0,0.0674875,0.00455457
GBM_3_AutoML_20190423_030857,1,2.05263e-17,0,1.30918e-15,1.71395e-30
GBM_2_AutoML_20190423_030857,1,1.16707e-17,0,6.78744e-16,4.60694e-31
StackedEnsemble_BestOfFamily_AutoML_20190423_030857,1,0.00149454,0,0.00161258,2.60041e-06
DeepLearning_1_AutoML_20190423_030857,1,0.000212221,0,0.00369512,1.36539e-05
GBM_grid_1_AutoML_20190423_030857_model_2,1,0.633817,0,0.469476,0.220408
StackedEnsemble_AllModels_AutoML_20190423_030857,1,0.00102106,0,0.00106645,1.13731e-06


In [17]:
aml1_leaderboard_df=aml1.leaderboard.as_data_frame()
model_set=aml1_leaderboard_df['model_id']
model_set

0             GLM_grid_1_AutoML_20190423_030857_model_1
1                          GBM_5_AutoML_20190423_030857
2                          GBM_1_AutoML_20190423_030857
3             GBM_grid_1_AutoML_20190423_030857_model_1
4                          GBM_3_AutoML_20190423_030857
5                          GBM_2_AutoML_20190423_030857
6     StackedEnsemble_BestOfFamily_AutoML_20190423_0...
7                 DeepLearning_1_AutoML_20190423_030857
8             GBM_grid_1_AutoML_20190423_030857_model_2
9      StackedEnsemble_AllModels_AutoML_20190423_030857
10                         DRF_1_AutoML_20190423_030857
11                         GBM_4_AutoML_20190423_030857
12                         XRT_1_AutoML_20190423_030857
13            GBM_grid_1_AutoML_20190423_030857_model_3
Name: model_id, dtype: object

In [ ]:
# meta data
metadata = meta_data(runid,server_path,data_path,test_path,model_path,target,run_time,classification,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(metadata)

In [ ]:
metadata = json.dumps(metadata)

In [ ]:
with open('metadata.json', 'w') as fp:
    json.dump(metadata, fp)

In [ ]:
df1 = lb1.as_data_frame()

In [ ]:
df1.to_csv("200leaderboard.csv")

In [18]:
model1 = h2o.get_model(lb1[0,'model_id'])

In [ ]:
model1 = json.dumps(model1)

In [ ]:
with open('model1-GBM_1.json', 'w') as fp:
    json.dump(model1, fp)

In [ ]:
model2 = h2o.get_model(lb1[1,'model_id'])
model2 = model2.params
with open('model2-GBM_4.json', 'w') as fp:
    json.dump(model2, fp)

In [ ]:
model3 = h2o.get_model(lb1[2,'model_id'])
model3 = model3.params
with open('model3-GBM_3.json', 'w') as fp:
    json.dump(model3, fp)

In [ ]:
model4 = h2o.get_model(lb1[3,'model_id'])
model4 = model4.params
with open('model4-GBM_grid_1_2.json', 'w') as fp:
    json.dump(model4, fp)

In [ ]:
model5 = h2o.get_model(lb1[4,'model_id'])
model5 = model5.params
with open('model5-GLM_grid_1_1.json', 'w') as fp:
    json.dump(model5, fp)

In [ ]:
model6 = h2o.get_model(lb1[5,'model_id'])
model6 = model6.params
with open('model6-GLM_grid_1_4.json', 'w') as fp:
    json.dump(model6, fp)

In [ ]:
model7 = h2o.get_model(lb1[6,'model_id'])
model7 = model7.params
with open('model7-GBM_5.json', 'w') as fp:
    json.dump(model7, fp)

In [ ]:
model8 = h2o.get_model(lb1[7,'model_id'])
model8 = model8.params
with open('model8-GBM_grid_1_3.json', 'w') as fp:
    json.dump(model8, fp)

In [ ]:
model9 = h2o.get_model(lb1[8,'model_id'])
model9 = model9.params
with open('model9-StackedEnsemble_AllModels.json', 'w') as fp:
    json.dump(model9, fp)

In [ ]:
model10 = h2o.get_model(lb1[9,'model_id'])
model10 = model10.params
with open('model10-DeepLearning_1.json', 'w') as fp:
    json.dump(model10, fp)

In [ ]:
model11 = h2o.get_model(lb1[10,'model_id'])
model11 = model11.params
with open('model11-DRF_1.json', 'w') as fp:
    json.dump(model11, fp)

In [ ]:
model12 = h2o.get_model(lb1[11,'model_id'])
model12 = model12.params
with open('model12-GBM_2.json', 'w') as fp:
    json.dump(model12, fp)

In [ ]:
model13 = h2o.get_model(lb1[12,'model_id'])
model13 = model13.params
with open('model13-GBM_grid_1_1.json', 'w') as fp:
    json.dump(model13, fp)

In [ ]:
model14 = h2o.get_model(lb1[13,'model_id'])
model14 = model14.params
with open('model14-StackedEnsemble_BestOfFamily.json', 'w') as fp:
    json.dump(model14, fp)

In [ ]:
model15 = h2o.get_model(lb1[14,'model_id'])
model15 = model15.params
with open('model15-DeepLearning_grid_1_2.json', 'w') as fp:
    json.dump(model15, fp)

In [ ]:
model16 = h2o.get_model(lb1[15,'model_id'])
model16 = model16.params
with open('model16-GBM_grid_1_5.json', 'w') as fp:
    json.dump(model16, fp)

In [ ]:
model17 = h2o.get_model(lb1[16,'model_id'])
model17 = model17.params
with open('model17-DeepLearning_grid_1_1.json', 'w') as fp:
    json.dump(model17, fp)

In [ ]:
model18 = h2o.get_model(lb1[17,'model_id'])
model18 = model18.params
with open('model18-XRT_1.json', 'w') as fp:
    json.dump(model18, fp)

In [ ]:
model19 = h2o.get_model(lb1[18,'model_id'])
model19 = model19.params
with open('model19-DeepLearning_grid_1_3.json', 'w') as fp:
    json.dump(model19, fp)

In [ ]:
model20 = h2o.get_model(lb1[19,'model_id'])
model20 = model20.params
with open('model20.json', 'w') as fp:
    json.dump(model20, fp)

In [ ]:
# setup autoML
# min_mem_size=6 
run_time=400
aml2 = H2OAutoML(max_runtime_secs=run_time)

In [ ]:
model_start_time = time.time()
aml2.train(x=X,y=y,training_frame=df2)  # Change training_frame=train

In [ ]:
# view leaderboard
lb2 = aml2.leaderboard
lb2.head(500)

In [ ]:
# setup autoML
# min_mem_size=6 
run_time=600
aml3 = H2OAutoML(max_runtime_secs=run_time)

In [ ]:
model_start_time = time.time()
aml3.train(x=X,y=y,training_frame=df2)  # Change training_frame=train

In [ ]:
lb3 = aml3.leaderboard
lb3.head(500)

In [ ]:
# setup autoML
# min_mem_size=6 
run_time=800
aml4 = H2OAutoML(max_runtime_secs=run_time)

In [ ]:
model_start_time = time.time()
aml4.train(x=X,y=y,training_frame=df2)  # Change training_frame=train

In [ ]:
lb4 = aml4.leaderboard
lb4.head(500)

In [ ]:
# setup autoML
# min_mem_size=6 
run_time=999
aml5 = H2OAutoML(max_runtime_secs=run_time)

In [ ]:
model_start_time = time.time()
aml5.train(x=X,y=y,training_frame=df2)  # Change training_frame=train

In [ ]:
lb5 = aml5.leaderboard
lb5.head(500)